In [1]:
!pip install pandas

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --------------------------- ------------ 7.9/11.5 MB 39.4 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 32.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------------------------------ --------- 9.7/12.6 MB 47.5 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 41.2 MB/s eta 0:00:00
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd


df = pd.read_csv('tmdb.csv')
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [3]:
import pandas as pd
import json


def collapse_genres(j):
  genres = []
  ar = json.loads(j)
  for a in ar:
    genres.append(a.get("name"))
  return " ".join(sorted(genres))


def combine_features(row):
  try:
    return row['overview']+" "+row["genres_name"]
  except:
    print ("Error:", row)


def process_tmdb_csv(input_file, output_file):
  """
  Processes a TMDB movies CSV file to create a Vespa-compatible JSON format.

  This function reads a CSV file containing TMDB movie data, processes the data to
  generate new columns for text search, and outputs a JSON file with the necessary
  fields (`put` and `fields`) for indexing documents in Vespa.

  Args:
    input_file (str): The path to the input CSV file containing the TMDB movies data.
                      Expected columns are 'id', 'original_title', 'overview', and 'genres'.
    output_file (str): The path to the output JSON file to save the processed data in
                       Vespa-compatible format.

  Workflow:
    1. Reads the CSV file into a Pandas DataFrame.
    2. Processes the 'genres' column, extracting genre names into a new 'genres_name' column.
    3. Fills missing values in 'original_title', 'overview', and 'genres_name' columns with empty strings.
    4. Creates a "text" column that combines specified features using the `combine_features` function.
    5. Selects and renames columns to match required Vespa format: 'doc_id', 'title', and 'text'.
    6. Constructs a JSON-like 'fields' column that includes the record's data.
    7. Creates a 'put' column based on 'doc_id' to uniquely identify each document.
    8. Outputs the processed data to a JSON file in a Vespa-compatible format.

  Returns:
    None. Writes the processed DataFrame to `output_file` as a JSON file.

  Notes:
    - The function requires the helper function `combine_features` to be defined, which is expected to combine text features for the "text" column.
    - Output JSON file is saved with `orient='records'` and `lines=True` to create line-delimited JSON.

  Example Usage:
    >>> process_tmdb_csv("tmdb_movies.csv", "output_vespa.json")
  """
  movies = pd.read_csv(input_file)
  movies['genres_name'] = movies.apply(lambda x: collapse_genres(x.genres), axis=1)
  for f in ['original_title','overview','genres_name']:
    movies[f] = movies[f].fillna('')

  movies["text"] = movies.apply(combine_features,axis=1)
  # Select only 'id', 'original_title', and 'text' columns
  movies = movies[['id', 'original_title', 'text']]
  movies.rename(columns={'original_title': 'title', 'id': 'doc_id'}, inplace=True)

  # Create 'fields' column as JSON-like structure of each record
  movies['fields'] = movies.apply(lambda row: row.to_dict(), axis=1)

  # Create 'put' column based on 'doc_id'
  movies['put'] = movies['doc_id'].apply(lambda x: f"id:hybrid-search:doc::{x}")

  df_result = movies[['put', 'fields']]
  print(df_result.head())
  df_result.to_json(output_file, orient='records', lines=True)


process_tmdb_csv("tmdb.csv", "clean_tmdb.jsonl")

                            put  \
0   id:hybrid-search:doc::19995   
1     id:hybrid-search:doc::285   
2  id:hybrid-search:doc::206647   
3   id:hybrid-search:doc::49026   
4   id:hybrid-search:doc::49529   

                                              fields  
0  {'doc_id': 19995, 'title': 'Avatar', 'text': '...  
1  {'doc_id': 285, 'title': 'Pirates of the Carib...  
2  {'doc_id': 206647, 'title': 'Spectre', 'text':...  
3  {'doc_id': 49026, 'title': 'The Dark Knight Ri...  
4  {'doc_id': 49529, 'title': 'John Carter', 'tex...  


In [4]:
!pip3 install --ignore-installed vespacli

   ---------------------------------------- 0.0/47.7 MB ? eta -:--:--
   ------ --------------------------------- 8.1/47.7 MB 41.1 MB/s eta 0:00:01
   --------------- ------------------------ 18.9/47.7 MB 45.5 MB/s eta 0:00:01
   ------------------------ --------------- 29.1/47.7 MB 46.4 MB/s eta 0:00:01
   -------------------------------- ------- 38.3/47.7 MB 45.3 MB/s eta 0:00:01
   ---------------------------------------  46.9/47.7 MB 44.5 MB/s eta 0:00:01
   ---------------------------------------- 47.7/47.7 MB 41.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!vespa config set target local

In [6]:
!vespa status deploy --wait 300

Deploy API at http://127.0.0.1:19071 is ready


Waiting up to 5m0s for deploy API...


In [9]:
!vespa deploy --wait 300 app


Success: Deployed 'app' with session ID 3


Waiting up to 5m0s for deploy API...
Waiting up to 5m0s for deployment to converge...
Waiting up to 5m0s for cluster discovery...
Waiting up to 5m0s for container default...


In [10]:
!vespa feed -t http://localhost:8080 clean_tmdb.jsonl


{
  "feeder.operation.count": 4803,
  "feeder.seconds": 51.460,
  "feeder.ok.count": 4803,
  "feeder.ok.rate": 93.334,
  "feeder.error.count": 0,
  "feeder.inflight.count": 0,
  "http.request.count": 4803,
  "http.request.bytes": 1670890,
  "http.request.MBps": 0.032,
  "http.exception.count": 0,
  "http.response.count": 4803,
  "http.response.bytes": 429878,
  "http.response.MBps": 0.008,
  "http.response.error.count": 0,
  "http.response.latency.millis.min": 42,
  "http.response.latency.millis.avg": 245,
  "http.response.latency.millis.max": 1300,
  "http.response.code.counts": {
    "200": 4803
  }
}


In [11]:
!pip install pyvespa


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached cryptography-43.0.3-cp39-abi3-win_amd64.whl.metadata (5.4 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached cffi-1.17.1-cp313-cp313-win_amd64.whl.metadata (1.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached charset_normalizer-3.4.0-cp313-cp313-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-win_amd64.whl.metadata (4.1 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached cryptography-43.0.3-cp39-abi3-win_amd64.whl (3.1 MB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached jinja2-3.1.4-py3-none-any.whl (133

In [13]:
#
# https://github.com/vespa-engine/sample-apps/blob/master/news/src/python/user_search.py
# https://docs.vespa.ai/en/tutorials/news-5-recommendation.html
#

# pip install pyvespa
import pandas as pd
from vespa.application import Vespa
from vespa.io import VespaResponse, VespaQueryResponse


def display_hits_as_df(response: VespaQueryResponse, fields) -> pd.DataFrame:
    records = []
    for hit in response.hits:
        record = {}
        for field in fields:
            record[field] = hit["fields"][field]
        records.append(record)
    return pd.DataFrame(records)


def keyword_search(app, search_query):
    query = {
        "yql": "select * from sources * where userQuery() limit 5",
        "query": search_query,
        "ranking": "bm25",
    }
    response = app.query(query)
    return display_hits_as_df(response, ["doc_id", "title"])


def semantic_search(app, query):
    query = {
        "yql": "select * from sources * where ({targetHits:100}nearestNeighbor(embedding,e)) limit 5",
        "query": query,
        "ranking": "semantic",
        "input.query(e)": "embed(@query)"
    }
    response = app.query(query)
    return display_hits_as_df(response, ["doc_id", "title"])


def get_embedding(doc_id):
    query = {
        "yql" : f"select doc_id, title, text, embedding from content.doc where doc_id contains '{doc_id}'",
        "hits": 1
    }
    result = app.query(query)
    
    if result.hits:
        return result.hits[0]
    return None


def query_movies_by_embedding(embedding_vector):
    query = {
        'hits': 5,
        'yql': 'select * from content.doc where ({targetHits:5}nearestNeighbor(embedding, user_embedding))',
        'ranking.features.query(user_embedding)': str(embedding_vector),
        'ranking.profile': 'recommendation'
    }
    return app.query(query)


# Replace with the host and port of your local Vespa instance
app = Vespa(url="http://localhost", port=8080)

query = "Harry Potter and the Half-Blood Prince"

df = keyword_search(app, query)
print(df.head())

df = semantic_search(app, query)
print(df.head())

emb = get_embedding("767")
results = query_movies_by_embedding(emb["fields"]["embedding"])
df = display_hits_as_df(results, ["doc_id", "title", "text"])
print(df.head())

  doc_id                                     title
0    767    Harry Potter and the Half-Blood Prince
1    671  Harry Potter and the Philosopher's Stone
2    674       Harry Potter and the Goblet of Fire
3    673  Harry Potter and the Prisoner of Azkaban
4  13967                               Miss Potter
   doc_id                                      title
0     767     Harry Potter and the Half-Blood Prince
1     675  Harry Potter and the Order of the Phoenix
2     672    Harry Potter and the Chamber of Secrets
3     674        Harry Potter and the Goblet of Fire
4  168705                                 BloodRayne
  doc_id                                      title  \
0    767     Harry Potter and the Half-Blood Prince   
1    675  Harry Potter and the Order of the Phoenix   
2    672    Harry Potter and the Chamber of Secrets   
3    671   Harry Potter and the Philosopher's Stone   
4    674        Harry Potter and the Goblet of Fire   

                                             